# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [4]:
#Your code here
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt

data=pd.read_csv('homepage_actions.csv')
data


,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view
8185,2017-01-18 10:01:09.026482,458115,experiment,view
8186,2017-01-18 10:08:51.588469,505451,control,view


In [5]:
data.info()
data.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


,id
count,8188.000000
mean,564699.749878
std,219085.845672
min,182988.000000
25%,373637.500000
50%,566840.500000
75%,758078.000000
max,937217.000000


In [6]:
#How many viewers also clicked?
clickers = set(data[data.action=='click']['id'])
viewers = set(data[data.action=='view']['id'])

print("Number of viewers: ",len(viewers))
print("Number of clickers: ",len(clickers))

#viewers minus the clickers(viewers who did not click)
print("Number of Viewers who didn't click: ",len(viewers-clickers))

#subtracts the viewers set from the clickers set, which gives the set of user IDs for users who clicked on the website but did not view it
print("Number of Clickers who didn't view: ",len(clickers-viewers))


Number of viewers:  6328
Number of clickers:  1860
Number of Viewers who didn't click:  4468
Number of Clickers who didn't view:  0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [7]:
#Your code here
data['count'] = 1
data.head()


,timestamp,id,group,action,count
0,2016-09-24 17:42:27.839496,804196,experiment,view,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,1
3,2016-09-24 19:59:02.646620,671993,control,view,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,1


In [8]:
#Convert clicks into a binary variable on a user-by-user-basis
control = data[data.group=='control'].pivot(index='id', columns='action', values='count')

experiment = data[data.group=='experiment'].pivot(index='id', columns='action', values='count')
#The pivot function is applied on the resulting subset, with the "id" column being used as the index, 
#"action" column as the columns, and "count" column as the values.


print("Viewers:  Control: {}  Experiment: {}" .format(len(control), len(experiment)))
print("Clickers:  Control: {}  Experiment: {}" .format(control.click.sum(), experiment.click.sum()))
print("Average click rate:  Control: {}  Experiment: {}" .format(control.click.mean(), experiment.click.mean()))
print("Average view rate:  Control: {}  Experiment: {}" .format(control.view.mean(), experiment.view.mean()))

control.head()

Viewers:  Control: 3332  Experiment: 2996
Clickers:  Control: 932.0  Experiment: 928.0
Average click rate:  Control: 1.0  Experiment: 1.0
Average view rate:  Control: 1.0  Experiment: 1.0


action,click,view
id,,
182994,1.0,1.0
183089,NaN,1.0
183248,1.0,1.0
183515,NaN,1.0
183524,NaN,1.0


In [9]:
control = control.fillna(value=0)
experiment = experiment.fillna(value=0)


In [10]:
control.head(10)

action,click,view
id,,
182994,1.0,1.0
183089,0.0,1.0
183248,1.0,1.0
183515,0.0,1.0
183524,0.0,1.0
183576,0.0,1.0
183617,1.0,1.0
184212,1.0,1.0
184390,0.0,1.0


In [11]:
experiment.head(10)

action,click,view
id,,
182988,0.0,1.0
183136,0.0,1.0
183141,1.0,1.0
183283,0.0,1.0
183389,0.0,1.0
183850,0.0,1.0
183938,1.0,1.0
183988,0.0,1.0
184368,0.0,1.0


In [27]:
len_exp_clicks = len(experiment.click)
len_exp_clicks

2996

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [26]:
#Your code here
ctr = control.click.mean()
ctr

0.2797118847539016

In [28]:
expected_number_of_clicks_for_the_experiment_group = ctr * len_exp_clicks
expected_number_of_clicks_for_the_experiment_group

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [35]:
#Your code here
n = len(experiment)
p = ctr
var = n * p * (1-p)
std = np.sqrt(var)

std

24.568547907005815

In [39]:
#z_score
expected_exp_clicks = experiment.click.sum()
z_score = (expected_exp_clicks - expected_number_of_clicks_for_the_experiment_group) / std

z_score

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [40]:
#Your code here
import scipy.stats as stats

p_value = 1 - stats.norm.cdf(3.6625360854823588)
p_value

0.00012486528006949715

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.